# **Simulazioni MD con il modulo CMS: CrystalStructure**

---

## Costruire un oggetto `CrystalStructure`

In `CMS`, un cristallo è rappresentato da un oggetto [CrystalStructure](../../CMS/MolecularDynamics/CrystalStructure.py), che può essere istanziato a partire da un file .txt contenente le coordinate degli atomi, ad esempio: [fcc100a256.txt](../../../data/pos-fcc100-random/fcc100a256.txt), dove 256 indica il numero `N` di atomi.

In [1]:
from CMS.MolecularDynamics.CrystalStructure import CrystalStructure
cristallo = CrystalStructure.from_file('../../../data/pos-fcc100-random/fcc100a256.txt')

---

## Contare il numero dei vicini di ciascun atomo

### Il raggio di _cutoff_ $R_C$

Tramite i metodi `.set_R_C()` il campo `R_C` di un oggetto CrystalStructure. `R_C` è la distanza entro la quale due atomi vengono considerati tra loro vicini. \
Il metodo `.find_neighbours()` si occupa di calcolare ogni distanza interatomica, contando il numero di vicini per ciascun atomo.
<img src="../images/1-CrystalStructure/cutoff-radius.png" 
     style="display: block; margin-left: auto; margin-right: auto; margin-top: 5px; width: 300px;">

In [ ]:
cristallo.set_R_C(4.5)  # in Angstrom
cristallo.find_neighbours()

Quando viene chiamato, `.find_neighbours()` memorizza i risultati in tre attributi:
<div align="center">

| Attributo | Dimensione | Contenuto |
| :--- | :--- | :--- |
| **`num_neighbours`** | $(N)$ | Array che contiene, nell'$i$-esima entrata, il **numero** di particelle considerate vicine dell'atomo $i$. |
| **`neighbour_list`** | $(N \times N)$* | Matrice che contiene, nell'$i$-esima riga, l'array degli **indici** dei vicini dell'atomo $i$. |
| **`distance_matrix`** | $(N \times N)$ | Matrice che contiene _tutte e solamente_ le **distanze** interatomiche tra atomi vicini. |
</div>

*`NxN` poiché, in assenza di cutoff, ogni atomo è considerato vicino per ciascun altro.

---

## Le condizioni al contorno

Con il metodo `CrystalStructure.set_pbc()` è possibile imporre delle condizioni di periodicità ai bordi del cristallo, stabilendo le dimensioni della cella $L_x, L_y$ e $L_z$ e ripetendola come un'unità periodica. L'assunzione è che la cella di simulazione sia una porzione rappresentativa di un _bulk_ infinito, cioè che in ogni altro volume della stessa dimensione $L_x \times L_y \times L_z$ il comportamento sia lo stesso. In questo modo, atomi molto distanti all'interno della cella, come verde $v$ e blu $b$ nell'immagine, possono interagire tra loro attraverso la loro immagine: nonostante $r_{b,v} > R_C$, esiste un'immagine di $v$ entro il raggio di _cutoff_ per $b$.

<img src="../images/1-CrystalStructure/periodic-boundary-conditions.png" 
     style="display: block; margin-left: auto; margin-right: auto; margin-top: 5px; width: 300px;">

`.set_pbc()` richiede in argomento una `tuple: (float, float, float)`, cioè una tripla `(Lx, Ly, Lz)`.

In [ ]:
pbc = (Lx, Ly, Lz)
cristallo.set_pbc(pbc)

#### Problemi

1. Il comportamento della cella unitaria è completamente identico nelle sue immagini: ogni atomo percorre le stesse traiettorie nelle celle replicate, c'è correlazione totale;
2. Un atomo può interagire con l'immafine di se stesso, se il raggio di cutoff è maggiore delle dimensioni della cella;
3. Gli effetti a lungo raggio (e.g. difetti puntuali) non sono replicabili, a meno di prendere celle sufficientemente grandi (vedere immagine).

<img src="../images/1-CrystalStructure/periodic-boundary-conditions-problems.png" 
     style="display: block; margin-left: auto; margin-right: auto; margin-top: 5px; width: 300px;">

#### Default

Di default, l'attributo `.pbc` è una tripla di infiniti `(np.inf, np.inf, np.inf)`; ciò è equivalente ad avere una cella isolata: non può esistere immagine di un atomo della cella a distanza inferiore di $R_C$ da qualunque altro. È possibile sfruttare questo principio per costruire superfici periodiche in due sole direzioni, ad esempio impostando `(Lx, Ly, np.inf)`. In realtà, se $Lz = Na + \Delta$, è sufficiente che $\Delta >> R_C$, poiché - vedere l'immagine - è come aggiungere uno spazio vuoto tra la cella simulata e le sue immagini.

<img src="../images/1-CrystalStructure/periodic-boundary-condition-settings.png" 
     style="display: block; margin-left: auto; margin-right: auto; margin-top: 5px; width: 400px;">

---

## _Speedup_: la cella di Verlet

> ⚠️ Per maggiori informazioni sulle simulazioni di dinamica, vedere il tutorial [4-CrystalDynamics](4-CrystalDynamics.ipynb).

Una delle operazioni più costose - in termini computazionali - di una simulazione di dinamica molecolare, è il calcolo dei vicini: `find_neighbours()` calcola tutte le distanze interatomiche e ciò richiede $N\times N$ operazioni. Per velocizzare le simulazioni, si possono sfruttare le _Verlet cages_. L'idea è definire una distanza $R_V > R_C$ entro la quale gli atomi vengono annoverati come 'vicini' ma non sono considerati nel computo dell'energia potenziale. In breve, le particelle nella regione tra $R_C$ e $R_V$ sono degli 'osservati speciali', che potrebbero finire nella sfera d'influenza di un altro atomo. Fintanto però che rimangono nella _Verlet cage_, senza oltrepassare $R_C$, non c'è necessità di ricalcolare i vicini utilizzando `.find_neighbours()`: si possono riutilizzare le liste già prodotte in precedenza.

<img src="../images/1-CrystalStructure/verlet-cage.png" 
     style="display: block; margin-left: auto; margin-right: auto; margin-top: 5px; width: 500px;">

Se viene impostato un _raggio di Verlet_ $R_V$ tramite `.set_R_V()` (si consiglia $R_V\simeq R_C+0.5\ \AA$), la simulazione applicherà la seguente logica:

<img src="../images/1-CrystalStructure/verlet-cage-algorithm.png" 
     style="display: block; margin-left: auto; margin-right: auto; margin-top: 5px; width: 400px;">

all'esecuzione di `.find_neighbours()` viene registrata, e considerata come **riferimento**, la posizione di ciascun atomo. Ad ogni timestep della simulazione gli atomi vengono spostati, secondo i principi in [4-CrystalDynamics](4-CrystalDynamics.ipynb), e viene calcolata la differenza tra la posizione attuale e quella di riferimento; quando **almeno per un atomo** questa è superiore a metà della grandezza della regione di Verlet, vengono ricalcolate _tutte_ le distanze interatomiche e ricostruite _da zero_ le liste dei vicini. Se invece tale condizione non si realizza, vengono aggiornate solamente le distanze interatomiche tra i vicini già determinati; ciò è compito del metodo `.only_neighbours_distance()`. In un normale workflow di `MD`, non è necessario chiamare manualmente la funzione.

L'implementazione corretta di $R_V$ può portare a una significativa riduzione del tempo di simulazione, facendo avvicinare il costo computazionale da quadratico nel numero di atomi $\mathcal{O}(N\times N)$ a lineare $\mathcal{O}(N)$, come dimostrato in [Verlet_Cages_Test](../../notebooks/Analisi-Varie/Verlet_Cages_Test.ipynb):

<img src="../images/1-CrystalStructure/verlet-cage-proof.png" 
     style="display: block; margin-left: auto; margin-right: auto; margin-top: 5px; width: 500px;">